# Calculate magnitudes of the stars

Written by Evgenii N.

This code calculates the magnitudes of stars.

## Prerequisite code

In [1]:
# Import libraries that we will use later in this notebook
import matplotlib.pyplot as plt

# Make images non-blurry on high pixel density screens
%config InlineBackend.figure_format = 'retina'


def set_plot_style():
    """Set global style"""

    # Title size
    plt.rcParams['axes.titlesize'] = 17

    # Axes label size
    plt.rcParams['axes.labelsize'] = 15

    # Tick label size
    plt.rcParams['xtick.labelsize'] = 13
    plt.rcParams['ytick.labelsize'] = 13
    
    # Legend text size
    plt.rcParams['legend.fontsize'] = 13

    # Grid color
    plt.rcParams['grid.color'] = '#cccccc'

    # Define plot size
    plt.rcParams['figure.figsize'] = [12, 8]

    # Marker size
    plt.rcParams['lines.markersize'] = 10
    
    
set_plot_style()

In [2]:
print("We are done!")

We are done!
